<a href="https://colab.research.google.com/github/manoj-b-22/IDS-in-IoV/blob/main/code/LSTM-inter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
scaler = StandardScaler()
encoder = LabelEncoder()

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/train-intervehicle.csv',chunksize=500000)
test = pd.read_csv('/content/drive/MyDrive/test-intervehicle.csv')

In [ ]:
model = Sequential()
model.add(LSTM(100,input_shape=(len(test.columns)-1,1)))
model.add(Dense(1))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 100)               40800     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for chunk in train:
  train_y = chunk.pop('Label')
  train_x = chunk
  train_x = scaler.fit_transform(train_x)
  train_y = encoder.fit_transform(train_y)
  model.fit(train_x,train_y,batch_size=100)

6161/6161 [==============================] - 30s 5ms/step - loss: 7.8231 - accuracy: 0.4928


In [ ]:
test_y = test.pop('Label')
test_x = test

In [ ]:
test_x = scaler.transform(test_x)
test_y = encoder.transform(test_y)

In [ ]:
score,accuracy = model.evaluate(test_x,test_y)
print('Score',score,"Accuracy",accuracy)

79666/79666 [==============================] - 316s 4ms/step - loss: 7.8270 - accuracy: 0.4926
Score 7.8269524574279785 Accuracy 0.4925849735736847
